In [27]:
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search,uniform_cost_search

## PROBLEMA COMPLETO CON 2 ASCENSORES

In [28]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError
        
    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return True



    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        
        for i in range(len(state[0])):
            if self.goal[i] != state[0][i]:
                return False 
        return True
        #if isinstance(self.goal, list):
         #   return is_in(state, self.goal)
        #else:
         #   return state[0] == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

In [29]:
class Ascensores(Problem):
   
    def __init__(self, initial, goal=((3,5,3,5,1))):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        #operadores: subir, bajar, parar

        ascensores_variable = estado[1][1]
        ascensores_constante = estado[1][0]
        
        #inicializar lista posiciones de los ascensores
        pos_ascensor= list()
        for i in range(len(ascensores_variable)):
            tupla = ascensores_variable[i]
            pos_ascensor.append(ascensores_variable[i][1]) #recorremos las tuplas de los ascensores y seleccionamos para cada uno su planta

        
     #pasajeros por ascensor
        pasajeros = list()
        for i in range(len(ascensores_variable)):
            pasajeros.append(ascensores_variable[i][0])
        
        
        #lista de personas aun por recoger para cada ascensor
        recoger = [ [] for i in range(len(ascensores_constante))]
        for i in range(len(list(estado[0]))):
            #comprobamos por cada ascensor(j)
             for j in range(len(ascensores_constante)):
                #ESTO ES UNA ÑAPA HAY QUE HACERLO BIEN
                if i not in pasajeros[0] and i not in pasajeros[1] and estado[0][i] != self.goal[i]:
                    #¿ if estado[0][i] < estado [1][0][0][3] ? Si la planta en la que estamos es < a la maxima del ascensor 0
                     if estado[0][i] < ascensores_constante[j][3] and estado[0][i] > ascensores_constante[j][2]:  
                        recoger[j].append(i)
                     elif(estado[0][i] == ascensores_constante[j][2] ):
                        if self.goal[i] > ascensores_constante[j][2]:
                            recoger[j].append(i)
                     elif(estado[0][i] == ascensores_constante[j][3] ):
                        if self.goal[i] < ascensores_constante[j][3]:
                            recoger[j].append(i)   
        
        #plantas destino de los pasajeros
        plantas = list()
        for i in range(len(pasajeros)):
            for j in range(len(pasajeros[i])):
                plantas.append(self.goal[pasajeros[i][j]])
            
    
        
        
        
        #lista de acciones
        accs = list()
        
        for i in range(len(ascensores_constante)):   #Recorremos la lista de ascensores para ver que acciones pueden hacer.      
            if pos_ascensor[i] < ascensores_constante[i][3]:
                #accs[i].append("Subir ascensor ", i)
                if(i == 0):
                    accs.append("Subir 0")
                else:
                    accs.append("Subir 1")
        
            if pos_ascensor[i] > ascensores_constante[i][2]:
                if(i == 0):
                    accs.append("Bajar 0")
                else:
                    accs.append("Bajar 1")
                 
            #no necesitamos comprobar los pasajeros del ascensor porque se actualizan en cada nodo
            #comprobamos ademas si hay alguien que recoger o si alguien ha llegaddo a su destino
            
            for j in range(len(recoger[i])):
                if (pos_ascensor[i] == estado[0][recoger[i][j]] and len(ascensores_variable[i][0]) < ascensores_constante[i][1]) or (pos_ascensor[i] in plantas):
                    if(i == 0):
                        accs.append("Parar 0")
                    else:
                        accs.append("Parar 1")
           
        return accs  
    
    def result(self,estado,accion):

        l = list(estado)
        l[1] = list(l[1])
        l[0] = list(l[0])
        l[1][0] = list(l[1][0])
        l[1][1] = list(l[1][1])
        l[1][0][0] = list(l[1][0][0])
        l[1][0][1] = list(l[1][0][1])
        l[1][1][0] = list(l[1][1][0])
        l[1][1][1] = list(l[1][1][1])
        l[1][1][0][0] = list(l[1][1][0][0])
        l[1][1][1][0] = list(l[1][1][1][0])

        
        
        
        asc1 = l[1][1][0]
        asc2 = l[1][1][1]
        plantas_pasaj = l[0]
        asc_const1 = l[1][0][0]
        asc_const2 = l[1][0][1]
    
       ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if accion == "Subir 0":
            asc1[2] = 1                   #actualizamos el estado del ascensor
            asc1[1] = asc1[1] + 1          #actualizamos la planta del ascensor
            for i in asc1[0]:             #actualizamos, para cada pasajero del ascensor, su planta
                plantas_pasaj[i] = asc1[1] 
         
        if accion == "Subir 1":
            asc2[2] = 1                   #actualizamos el estado del ascensor
            asc2[1] = asc2[1] + 1          #actualizamos la planta del ascensor
            for i in asc2[0]:             #actualizamos, para cada pasajero del ascensor, su planta
                plantas_pasaj[i] = asc2[1] 
            
            
        if accion == "Bajar 0":
            asc1[2] = -1                  #actualizamos el estado del ascensor
            asc1[1] = asc1[1] - 1          #actualizamos la planta del ascensor
            for i in asc1[0]:             #actualizamos, para cada pasajero del ascensor, su planta
                plantas_pasaj[i] = asc1[1] 
           
        
        if accion == "Bajar 1":
            asc2[2] = -1                  #actualizamos el estado del ascensor
            asc2[1] = asc2[1] - 1          #actualizamos la planta del ascensor
            for i in asc2[0]:             #actualizamos, para cada pasajero del ascensor, su planta
                plantas_pasaj[i] = asc2[1]
                
                
        if accion == "Parar 0":
            asc1[0] = [x for x in asc1[0] if not (self.goal[x] == asc1[0])]  #bajamos del ascensor a los que han llegado a su planta
            #lo mismo que a continuacion
            #for e in ascensor[1]:                                                  
                #if self.goal[0][e] == ascensor[0]:
                    #del(ascensor[1][e])
            if len(asc1[0]) < asc_const1[1]:                                   #subimos gente al ascensor si hay hueco
                for i in range(len(plantas_pasaj)) :
                    if (len(asc1[0]) < asc_const1[1]) and (self.goal[i] != estado[0][i]):     
                        if (plantas_pasaj[i] == asc1[1]) and (i not in asc1[0]) :
                            asc1[0].append(i)

          
        if accion == "Parar 1":
            asc2[0] = [x for x in asc2[0] if not (self.goal[x] == asc2[0])]  #bajamos del ascensor a los que han llegado a su planta
            #lo mismo que a continuacion
            #for e in ascensor[1]:                                                  
                #if self.goal[0][e] == ascensor[0]:
                    #del(ascensor[1][e])
            if len(asc2[0]) < asc_const2[1]:                                   #subimos gente al ascensor si hay hueco
                for i in range(len(plantas_pasaj)) :
                    if (len(asc2[0]) < asc_const2[1]) and (self.goal[i] != estado[0][i]):     
                        if (plantas_pasaj[i] == asc2[1]) and (i not in asc2[0]) :
                            asc2[0].append(i)

        
    
        #tuplificamos
        
        l[1][1][0] = asc1
        l[1][1][1] = asc2
        l[0] = plantas_pasaj 
        l[1][0][0] = asc_const1 
        l[1][0][1] = asc_const2                 


        l[1][1][0][0] = tuple(l[1][1][0][0])
        l[1][1][1][0] = tuple(l[1][1][1][0])
        l[1][0][0] = tuple(l[1][0][0])
        l[1][0][1] = tuple(l[1][0][1])
        l[1][1][0] = tuple(l[1][1][0])
        l[1][1][1] = tuple(l[1][1][1])
        l[1][0] = tuple(l[1][0])
        l[1][1] = tuple(l[1][1])
        l[1] = tuple(l[1])
        l[0] = tuple(l[0])
       
        return tuple(l)
    
    def h(self, node):
        " Return the heuristic value for a given state."
        return 

In [30]:
asc = Ascensores(((0,5,0,5,1), (((1,2,0,4), (1,2,4,8)), (((0,),0,0),((3,1),5,1)))))

asc.initial
print(asc.initial)
asc.actions(asc.initial)

((0, 5, 0, 5, 1), (((1, 2, 0, 4), (1, 2, 4, 8)), (((0,), 0, 0), ((3, 1), 5, 1))))


['Subir 0', 'Parar 0', 'Subir 1', 'Bajar 1']

In [31]:
asc.result(asc.initial, "Parar 1")

((0, 5, 0, 5, 1),
 (((1, 2, 0, 4), (1, 2, 4, 8)), (((0,), 0, 0), ((3, 1), 5, 1))))

## ESPECIFICACION DE LOS ALGORITMOS

In [32]:
class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion, index):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        #ALWAYS SOLVABLE :)
        return 1   


In [62]:
#heuristicas para el problema simplificado
#primera heurística: personas que no estan en destino
def h1(node):
    state = node.state
    goals = [3,3,3,0,2]
    count = 0
    for i in range(len(state[0])):
        if (goals[i] != state[0][i]):
            count = count + 1      
    return count



#segunda heurística: similar a la distancia M
def h2(node):
    state = node.state
    goals = [3,3,3,0,2]
    #goals = node.state.goal
    count = 0
    for i in range(len(state[0])):
        count = count + abs(goals[i] - state[0][i])

    return count



#tercera heurística: distancia M + (personas fuera de destino/capacidad ascensor) * huecos ascensor
def h3(node):
    state = node.state
    goals = [3,3,3,0,2]
    no_destino = 0
    count = 0
    count1 = 0
    count2 = 0
    
    for i in range(len(state[0])):
        if (goals[i] != state[0][i]):
            count = abs(goals[i] - state[0][i])
            no_destino = no_destino + 1
     
    count1 += (no_destino/2)*(state[1][0][0][1]-len(state[1][1][0][0]))
    count2 += (no_destino/2)*(state[1][0][1][1]-len(state[1][1][1][0]))
    count += max(count1,count2)
    return count





In [63]:
def resuelve_ascensores(estado_inicial, algoritmo, h=None):

    p8p=Problema_con_Analizados(Ascensores(estado_inicial))
    if p8p.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")




In [64]:
estado = Ascensores(((0,5,0,5,1), (((1,2,0,4), (1,2,4,8)), (((0,),0,0),((3,1),5,1)))))
breadth_first_graph_search(estado).solution()

#breadth_first_tree_search(estado).solution()
#depth_first_tree_search(estado).solution()

['Parar 0', 'Subir 0', 'Subir 0', 'Subir 0']

In [36]:
uniform_cost_search(estado).solution()

['Parar 0', 'Subir 0', 'Subir 0', 'Subir 0']

In [37]:
depth_first_graph_search(estado).solution()

['Bajar 1',
 'Parar 0',
 'Subir 0',
 'Subir 1',
 'Subir 1',
 'Bajar 1',
 'Subir 0',
 'Bajar 0',
 'Subir 1',
 'Subir 1',
 'Bajar 1',
 'Subir 0',
 'Subir 0',
 'Subir 1',
 'Subir 1',
 'Bajar 1',
 'Subir 0',
 'Bajar 0',
 'Bajar 1',
 'Bajar 1']

In [42]:
estado_inicial = ((0,5,0,5,1), (((1,2,0,4), (1,2,4,8)), (((0,),0,0),((3,1),5,1))))

In [43]:
%%time
resuelve_ascensores(estado_inicial,uniform_cost_search)

Solución: ['Parar 0', 'Subir 0', 'Subir 0', 'Subir 0']
Algoritmo: uniform_cost_search
Longitud de la solución: 4. Nodos analizados: 51
CPU times: user 4.95 ms, sys: 197 µs, total: 5.15 ms
Wall time: 5.05 ms


In [44]:
%%time
resuelve_ascensores(estado,depth_first_graph_search)

Solución: ['Bajar 1', 'Parar 0', 'Subir 0', 'Subir 1', 'Subir 1', 'Bajar 1', 'Subir 0', 'Bajar 0', 'Subir 1', 'Subir 1', 'Bajar 1', 'Subir 0', 'Subir 0', 'Subir 1', 'Subir 1', 'Bajar 1', 'Subir 0', 'Bajar 0', 'Bajar 1', 'Bajar 1']
Algoritmo: depth_first_graph_search
Longitud de la solución: 20. Nodos analizados: 34
CPU times: user 2.8 ms, sys: 111 µs, total: 2.91 ms
Wall time: 2.95 ms


In [45]:
%%time
resuelve_ascensores(estado_inicial,depth_first_tree_search)

KeyboardInterrupt: 

In [46]:
%%time
resuelve_ascensores(estado_inicial,breadth_first_tree_search)

Solución: ['Parar 0', 'Subir 0', 'Subir 0', 'Subir 0']
Algoritmo: breadth_first_tree_search
Longitud de la solución: 4. Nodos analizados: 124
CPU times: user 6.49 ms, sys: 399 µs, total: 6.89 ms
Wall time: 7 ms


In [47]:
%%time
resuelve_ascensores(estado_inicial,breadth_first_graph_search)

Solución: ['Parar 0', 'Subir 0', 'Subir 0', 'Subir 0']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 4. Nodos analizados: 45
CPU times: user 1.99 ms, sys: 83 µs, total: 2.08 ms
Wall time: 2.05 ms


In [48]:
%%time
resuelve_ascensores(estado_inicial,depth_limited_search)

Solución: ['Subir 0', 'Subir 0', 'Subir 0', 'Bajar 0', 'Bajar 0', 'Bajar 0', 'Parar 0', 'Subir 0', 'Subir 0', 'Subir 0']
Algoritmo: depth_limited_search
Longitud de la solución: 10. Nodos analizados: 5272
CPU times: user 68.2 ms, sys: 2.18 ms, total: 70.4 ms
Wall time: 71.6 ms


In [49]:
%%time
resuelve_ascensores(estado_inicial,iterative_deepening_search)

Solución: ['Parar 0', 'Subir 0', 'Subir 0', 'Subir 0']
Algoritmo: iterative_deepening_search
Longitud de la solución: 4. Nodos analizados: 173
CPU times: user 2.72 ms, sys: 80 µs, total: 2.8 ms
Wall time: 2.76 ms


In [65]:
%%time
resuelve_ascensores(estado_inicial,astar_search,h1)

Solución: ['Parar 0', 'Subir 0', 'Subir 0', 'Subir 0']
Algoritmo: astar_search
Heurística: h1
Longitud de la solución: 4. Nodos analizados: 30
CPU times: user 3.19 ms, sys: 92 µs, total: 3.28 ms
Wall time: 3.25 ms


In [66]:
%%time
resuelve_ascensores(estado_inicial,astar_search,h2)

Solución: ['Bajar 1', 'Parar 0', 'Subir 0', 'Subir 0', 'Subir 0', 'Subir 1']
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 6. Nodos analizados: 14
CPU times: user 1.36 ms, sys: 91 µs, total: 1.45 ms
Wall time: 1.42 ms


In [67]:
%%time
resuelve_ascensores(estado_inicial,astar_search,h3)

UnboundLocalError: local variable 'count1' referenced before assignment

In [59]:
resuelve_ascensores(estado_inicial,best_first_graph_search,h1)

Solución: ['Bajar 1', 'Parar 0', 'Subir 0', 'Subir 0', 'Subir 0', 'Subir 1']
Algoritmo: best_first_graph_search
Heurística: h1
Longitud de la solución: 6. Nodos analizados: 69


In [60]:
resuelve_ascensores(estado_inicial,best_first_graph_search,h2)

Solución: ['Bajar 1', 'Subir 0', 'Bajar 0', 'Parar 0', 'Subir 0', 'Subir 0', 'Subir 0', 'Subir 1']
Algoritmo: best_first_graph_search
Heurística: h2
Longitud de la solución: 8. Nodos analizados: 18


In [61]:
resuelve_ascensores(estado_inicial,best_first_graph_search,h3)

Solución: ['Parar 0', 'Subir 1', 'Bajar 1', 'Subir 0', 'Subir 0', 'Subir 0']
Algoritmo: best_first_graph_search
Heurística: h3
Longitud de la solución: 6. Nodos analizados: 51
